In [1]:

from transformers import pipeline
from transformers import AutoModelForSequenceClassification, AutoTokenizer


In [2]:

device = "cpu"


In [3]:
sequence = "I am going to france" 

label = ['travel', 'cooking', 'dancing']

In [4]:

nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')


In [5]:

premise    = sequence
hypothesis = f'This example is {label}.'



In [6]:
premise

'I am going to france'

In [7]:
hypothesis

"This example is ['travel', 'cooking', 'dancing']."

In [8]:
# run through model pre-trained on MNLI

x = tokenizer.encode(premise, hypothesis, return_tensors='pt', truncation_strategy='only_first')


C:\Users\user1\anaconda3\envs\huggingface\lib\site-packages\transformers\tokenization_utils_base.py:2390: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  FutureWarning,


In [9]:
x

tensor([[    0,   100,   524,   164,     7,  6664,  2389,     2,     2,   713,
          1246,    16, 47052, 28881,  3934,   128, 35190,   154,  3934,   128,
           417,  7710,   108,  8174,     2]])

In [12]:
logits = nli_model(x.to(device))[0]

In [13]:
logits

tensor([[-2.0327,  1.3776,  0.6796]], grad_fn=<AddmmBackward0>)

In [14]:

# we throw away "neutral" (dim 1) and take the probability of
# "entailment" (2) as the probability of the label being true 

entail_contradiction_logits = logits[:,[0,2]]
entail_contradiction_logits

tensor([[-2.0327,  0.6796]], grad_fn=<IndexBackward0>)

In [15]:
probs = entail_contradiction_logits.softmax(dim=1)

probs


tensor([[0.0623, 0.9377]], grad_fn=<SoftmaxBackward0>)

In [16]:
prob_label_is_true = probs[:,1]
prob_label_is_true

tensor([0.9377], grad_fn=<SelectBackward0>)